# Integral Control Analysis

# Preliminaries

## Imports

In [1]:
import collections
import tellurium as te
import sympy
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
import numpy as np
from common_python.sympy import sympyUtil as su
from common_python.ODEModel.ODEModel import ODEModel

## Constants

In [ ]:
def initSymbols():
    symbols = "dA A A_ref K_AN K_AM N K_PA K_MP rd_A rd_A1 K1 K2 K3 K4 K_A2 Kd_A Kd_M Kd_P K_Mp A M Mp dP P K_P k_0 M dM K_M rd_M k_0"
    su.addSymbols(symbols, dct=globals())

initSymbols()
EQUATION_VARS = [P, M, A]

In [ ]:
STATE_DCT = {
     A: K_AN * N - Kd_A * A * M,
     P: K_PA * A - k_0,
     M: K_MP * P - Kd_M * M,
}
SUBS = {Kd_A: 1, K_MP: 1, K_PA: 1, k_0: 2.0, Kd_M: 1, N: 1, K_AN: 1, Kd_M: 1, Kd_A: 1, Kd_P: 1, K_AM: 1}

# Helper Functions

In [ ]:
def mkSymbolStmt(symbolStr):
  """
  Creaes an executable statement that defines the blank separated symbols in the string.
  """
  symbols = symbolStr.split(" ")
  quotedStrs = ["%s" %s for s in symbols]
  argStrs = ", ".join(symbols)
  return "%s = sympy.symbols(%s)" % (argStrs, quotedStrs)

# Tests
variables = ["aa", "bbb", "fruit"]
symbolStr = " ".join(variables)
stmt = mkSymbolStmt(symbolStr)
exec(stmt)
for variable in variables:
  assert(variable in locals())
  delStmt = "del %s" % variable
  exec(delStmt)

In [ ]:
def evaluate(expression, subs):
    result = expression.copy()
    for symbol, value in subs.items():
        result = result.subs(symbol, value)
    return sympy.simplify(result)

# Tests
initSymbols()
dA = A + A * M
result = evaluate(dA, {A: 1})
assert(result == 1 + M)

In [ ]:
def initEquations():
  """
  Constructs equations for network A where M inhibits A
  """
  statements = [
      "dA = K_AN * N - Kd_A * A * M",
      "dP = K_PA * A - k_0 ",
      "dM = K_MP * P - Kd_M * M",
      "equationDct = {A: dA, P: dP, M: dM}",
  ]
  return "; ".join(statements)

exec(initEquations())

In [ ]:
def mkEquationVec(equationDct, equationVars=[P, M, A]):
    return sympy.Matrix([equationDct[s] for s in equationVars])

# Tests
exec(initEquations())
vec = mkEquationVec(equationDct)
assert(len(vec) == 3)
assert(isinstance(vec, sympy.Matrix))

In [ ]:
def calcJacobian(equationDct, fixedDct, equationVars=[P, M, A], **kwargs):
    """
    Calculates the Jacobian at the fixed point and evaluates it.
    
    Parameters
    ----------
    equationDct: key: Symbol, value: expression
    fixedDct: key: Symbol, value: expression/number
    varVec: vector of variable (symbols) that correspond to the equations
    kwargs: dict
        optional arguments for evaluate
        
    Returns
    -------
    sympy.Matrix
    """
    equationVec = mkEquationVec(equationDct, equationVars)
    # Calculate fixed points and eigenvectors
    jacobianMat = equationVec.jacobian(equationVars)
    jacobianMat = su.evaluate(jacobianMat, subs=fixedDct, isNumpy=False)
    if len(kwargs) > 0:
        jacobianMat = su.evaluate(jacobianMat, **kwargs)
    return jacobianMat

# Tests
exec(initEquations())
fixedDct = {A:101, M: 1001}
resultMat = calcJacobian(equationDct , fixedDct, isNumpy=False)
assert(resultMat[2, 2] == -1001.0 * Kd_A)
assert((resultMat.rows == 3) and (resultMat.cols ==3))

In [ ]:
# eqn: equations for model
# var: variable names
# fps: list-FixedPointInfo
# jac: jacobian
# eig: eigenvalues
# fp: list-dct of fixed points
ModelInfo = collections.namedtuple("ModelInfo", "eqn var fps")
FixedPointInfo = collections.namedtuple("FixedPointInfo", "jac eig fp")
def calcModelInfo(equationDct, equationVars=EQUATION_VARS, subs={}):
    """
    Construct various information for a model.
    
    Parameters
    ----------
    equationDct: dict
        key: state
        value: expression
    equationVars: list-symbol
        state variables
    subs: dict
    
    Returns
    -------
    ModelInfo
    """
    def mkFixedDcts(fixedPoints):
        def mkDct(points):
            if isinstance(points, dict):
                return points
            return {v: p for p, v in zip(points, equationVars)}
        #
        if isinstance(fixedPoints, dict):
            fixedDcts = [mkDct(fixedPoints)]
        else:
            fixedDcts = []
            for fixedPoint in fixedPoints:
                fixedDcts.append(mkDct(fixedPoint))
        return fixedDcts
    #
    modelInfos = []
    equationVec = mkEquationVec(equationDct, equationVars)
    # Calculate the fixed points
    fixedPoints = sympy.solve(equationVec, equationVars)
    fixedDcts = mkFixedDcts(fixedPoints)
    # Calculate the Jacobian
    fps = []
    for fixedDct in fixedDcts:
        jacobianMat = calcJacobian(equationDct, fixedDct, isNumpy=False)
        eigenValues = list(jacobianMat.eigenvals().keys())
        fps.append(FixedPointInfo(fp=fixedDct, jac=jacobianMat, eig=eigenValues))
    # Construct the ModelInfo
    return ModelInfo(eqn=equationDct, var=equationVars, fps=fps)

# Tests
exec(initEquations())
result = calcModelInfo(equationDct)
eigenvalues = [su.evaluate(e, subs=SUBS) for e in result.fps[0].eig]
assert(eigenvalues[0] < 0)
assert(len(eigenvalues) == 3)

# Implementation of integral feedback control in biological systems
## Reference
Authors: 
Somvanshi, Pramod R.
Patel, Anilkumar K.
Bhartiya, Sharad
Venkatesh, K. V.

Date: 2015

## Analysis

[Feedback Designs](https://drive.google.com/file/d/1GT2hfqxUAo8iiLutB4ccp_mMl59KB8OH/view?usp=sharing)

* $P$ - protein
* $A$ - signaling molecule
* $M$ - metabolite
* $N$ - disturbance
* $E = A_{ref} - A$
* $A_{ref} = \frac{k_0}{K_P}$

* $P \xrightarrow{k_0}\phi$
* $\phi \xrightarrow{K_M P} M$
* $A \xrightarrow{Kd_A A M} \phi$
* $N \xrightarrow{K_A} A$
* $\phi \xrightarrow{K_P A} P$

In [ ]:
NET_A = """

J1: P -> ; k_0
J3: -> P; K_P * A
J2: -> M; K_M * P
J2a: M ->; Kd_M * M
J4: A -> ; Kd_A * A * M
J5: $N -> A; K_A * N

$N = 1
P = 100
M = 1
A = 1
K_N = 1
k_0 = 1
K_M = 1
Kd_M = 1
K_P = 1
Kd_A = 1
K_A = 1
"""

rr = te.loada(NET_A)
rr.plot(rr.simulate(0, 1000, 100), title="N=%d" % rr.N)
print("Final A = %2.2f" % rr.A)

### Revised Network A

In [ ]:
# Have a source of A other than N
NET_A1 = """

J1: P -> ; k_0
J3: -> P; K_P * A
J2: -> M; K_M * P
J2a: M ->; Kd_M * M
J4: A -> ; Kd_A * A * M
J4a: -> A; k_1
J5: $N -> A; K_A * N

$N = 1
P = 1
M = 1
A = 1
K_N = 1
k_0 = 1
K_M = 1
Kd_M = 1
K_P = 1
Kd_A = 1
K_A = 1
k_1 = 1
"""

rr = te.loada(NET_A1)
rr.plot(rr.simulate(0, 30, 100), title="N=%d" % rr.N)
print("Final A = %2.2f" % rr.A)

### Changing $k_0$

Vary k_0 from 0.5 to 3.
Plot fixed points and how they change type.

In [ ]:
# Have a source of A other than N
NET_A1 = """

J1: P -> ; k_0
J3: -> P; K_P * A
J2: -> M; K_M * P
J2a: M ->; Kd_M * M
J4: A -> ; Kd_A * A * M
J4a: -> A; k_1
J5: $N -> A; K_A * N

$N = 1
P = 1
M = 1
A = 1
K_N = 1
k_0 = 1
K_M = 1
Kd_M = 1
K_P = 1
Kd_A = 1
K_A = 1
k_1 = 1
"""

rr = te.loada(NET_A1)
rr.plot(rr.simulate(0, 30, 100), title="N=%d" % rr.N)
print("Final A = %2.2f" % rr.A)

In [ ]:
K0_VALS = [0.5, 0.75, 1.0, 1.5, 2.0, 2.5]

In [ ]:
def plotk0(model, k0Vals=K0_VALS):
    numCol = 3
    fig, axes = plt.subplots(2, numCol, figsize=(12, 10))
    for idx, k_0 in enumerate(k0Vals):
        if idx < numCol:
            ax = axes[0, idx]
        else:
            ax = axes[1, idx - numCol]
        rr =  te.loada(model)
        rr.k_0 = k_0
        data = rr.simulate(0, 30, 100)
        xv = data[:, 0]
        columns = [s[1:-1] for s in data.colnames[1:]]
        for idx1, column in enumerate(columns):
            ax.plot(xv, data[:, idx1+1])
        ax.plot(xv, np.repeat(k_0, len(xv)), linestyle="--", color="black")
        if idx == 0:
            ax.legend(columns)
        if idx == numCol:
            ax.set_xlabel("time")
            ax.set_ylabel("value")
        ax.set_title("k_0=%2.2f" % k_0)
        ax.set_ylim([0, 8])
        
# Tests
plotk0(NET_A1)

In [ ]:
def calcEigenVals(xVec, dxVec, subs):
    """
    Calculates the eigenvectors of the linearized derivative matrix
    at the substituted values.
    
    Parameters
    ----------
    xVec: sympy.Mat N X 1
        symbols for states
    dxVec: sympy.Mat N X 1
        expressions for derivatives of state
    subs: dict
        substitutions to make for eigenvalue calculation
        key: symbol
        value: value to use for symbol
        
    Returns
    -------
    dict
        key: sympy.Matrix N X 1
            fixed point
        value: sympy.Matrix N X 1
            eigenvalues
    """
    jacobianMat = dxVec.jacobian(xVec)
    fixedPoints = sympy.solve(dxVec, xVec)
    dct = {}
    jacobianMat = evaluate(jacobianMat, subs)
    for fixedPoint in fixedPoints:
        numericFixedPoint = tuple([sympy.simplify(evaluate(f, subs)) for f in fixedPoint])
        fixedSubs = {s: v for s, v in zip(xVec, numericFixedPoint)}
        mat = evaluate(jacobianMat, fixedSubs)
        mat = evaluate(mat, subs)
        dct[numericFixedPoint] = sympy.Matrix(list(mat.eigenvals().keys()))
        dct[numericFixedPoint] = dct[numericFixedPoint].evalf()
    return dct

# Tests
initSymbols()
exec(initEquations())
xVec = sympy.Matrix([A, P, M])
dxVec = mkEquationVec(equationDct, equationVars=[A, P, M])
#dxVec = sympy.Matrix(equations)
dct = calcEigenVals(xVec, dxVec, SUBS)
assert(list(dct.keys()) == [(2.0, 0.5, 0.5)])

In [ ]:
# Analysis of fixed points

subs = dict(SUBS)
for val in K0_VALS:
    initSymbols()
    subs[k_0] = val
    xVec = sympy.Matrix([A, P, M])
    equationVec = mkEquationVec(equationDct, equationVars=xVec)
    dxVec = sympy.Matrix(equationVec)
    dct = calcEigenVals(xVec, dxVec, subs)
    print(dct)

## Network A

In [ ]:
modelA = ODEModel(STATE_DCT, isEigenvecs=False)

In [ ]:
for fp in modelA.fixedPoints:
    print(fp.valueDct)
    print(fp.getEigenvalues(subs=SUBS))
    print("\n")

## Network B

Network B1 has degrdation of $P$ that depends on $P$. It has two problems:
* Eigenvectors are a funxtion of $M$
* Fixed points are a function of $N$

In [ ]:
stateDct = dict(STATE_DCT)
stateDct[A] = K_AN * N - Kd_A*A + K_AM * M
stateDct[P] = -K_PA * A * M  + k_0

In [ ]:
modelB1 = ODEModel(stateDct, isEigenvecs=False)

In [ ]:
modelB1.jacobianMat

In [ ]:
modelB1.fixedPoints[0].valueDct[P]

In [ ]:
 modelB1.fixedPoints[0].valueDct[M]

In [ ]:
 modelB1.fixedPoints[0].valueDct[A]

Network B2 has degrdation of $P$ that does *not* depend on $P$.
* It has a fixed point for $A$ ad $k_0 / K_{PA}$.
* It has no dependency on the state variables, and so the dynamics are independent of the fixed point. (It is a linear system.)

In [ ]:
stateDct = dict(STATE_DCT)
stateDct[A] = K_AN * N - Kd_A*A + K_AM * M
stateDct[P] = -K_PA * A  + k_0

In [ ]:
modelB2 = ODEModel(stateDct, isEigenvecs=False)

In [ ]:
modelB2.jacobianMat

In [ ]:
modelB2.fixedPoints[0].valueDct[P]

In [ ]:
modelB2.fixedPoints[0].valueDct[M]

In [ ]:
modelB2.fixedPoints[0].valueDct[A]

In [ ]:
for fp in modelB2.fixedPoints:
    print(fp.getEigenvalues(subs=SUBS))

In [ ]:
# Have a source of A other than N
NET_B1 = """

J1: -> P ; k_0
J3: P -> ; K_PA * A * P
J2: -> M; K_MP * P
J2a: M ->; Kd_M * M
J4: A -> ; Kd_A * A
J4a: -> A; K_AM*M
J5: $N -> A; K_AN * N

$N = 0
P = 0
M = 0
A = 0
K_AN = 1
k_0 = 5
K_MP = 1
Kd_M = 1
K_PA = 1
Kd_A = 1
K_AM = 1
"""

rr = te.loada(NET_B1)
rr.plot(rr.simulate(0, 30, 100), title="N=%d" % rr.N)
print("Final A = %2.2f" % rr.A)

In [ ]:
plotk0(NET_B1)

In [ ]:
# Have a source of A other than N
NET_B2 = """

J1: -> P ; k_0
J3: P -> ; K_PA * A
J2: -> M; K_MP * P
J2a: M ->; Kd_M * M
J4: A -> ; Kd_A * A
J4a: -> A; K_AM*M
J5: $N -> A; K_AN * N

$N = 0
P = 0
M = 0
A = 0
K_AN = 1
k_0 = 5
K_MP = 1
Kd_M = 1
K_PA = 1
Kd_A = 1
K_AM = 1
"""

rr = te.loada(NET_B2)
rr.plot(rr.simulate(0, 30, 100), title="N=%d" % rr.N)
print("Final A = %2.2f" % rr.A)

In [ ]:
plotk0(NET_B2)

**Observations**
1. Can change $k_0$ without affect Jacobian. However, fixed point depends on $k_0$, and so dynamics are still impacted if there is a change in the setpoint.

## Network C

In [ ]:
# Design 1(c): Both M and E inhibit A. Again, need a constant influx so that steady states can be achieved.
exec(initEquations())
dA = - K_AN * N - Kd_A * M + K1
#
result = sympy.solve([dP, dM, dA], [P, M, A])
result

## Network D

In [ ]:
# Design 1(d): E inhibits A; A inhibits P; M activates A
exec(initEquations())
dA = K_AM * M - K_AN * N - Kd_A * A
dP = -K_PA * A + k_0  # Since A inhibits, need 0th order kinetics for creation of P
#
result = sympy.solve([dP, dM, dA], [P, M, A])
result

Can construct a table with cases defined by:
* E on A
* M on A
* A on P

Theme: Systematic design of integral control based on:
1. the placement of inhibit vs. activate and use of 0th order kinetics
1. which term(s) have 0th order kinetics

## Tracking for Architecture (a)

In [ ]:
exec(initEquations())
dP = K_P * A - k_0 * A_ref
result = sympy.solve([dP, dM, dA], [P, M, A])
result

In [ ]:
TRACKING_MODEL_A = """
->P; K_P*A
P -> A; K_P * P
A -> P; -KA_1 * A
A ->; KA * $M
// E -> A; KE * A

$M = 1
KP_1 = 1
KP_2 = 1
KA_1 = 1
KA = 1
KE = 1
K_P = 1
"""

rr = te.loada(TRACKING_MODEL_A)
rr.plot(rr.simulate())

# Signaling Cascades

In [24]:
su.addSymbols("A_1 A_2 A_3 A_4 T_1 T_2 T_3 T_4 K_1 K_2 K_3 K_4 k_1 k_2 k_3 k_4 S_0 S_1")

In [25]:
# State equations
stateDct = {
    A_1: K_1 * S_0 * (T_1 - A_1) - k_1 * A_1,
    A_2: K_2 * A_1 * (T_2 - A_2) - k_2 * A_2,
    A_3: K_3 * A_2 * (T_3 - A_3) - k_3 * A_3,
    A_4: K_4 * A_3 * (T_4 - A_4) - k_4 * A_4,
}
modelSig = ODEModel(stateDct, isEigenvecs=False)

In [26]:
fps = modelSig.fixedPoints
fps[0].valueDct[A_1]

K_1*S_0*T_1/(K_1*S_0 + k_1)

In [15]:
fps[0].valueDct[A_2]

K_1*K_2*S_0*T_1*T_2/(K_1*K_2*S_0*T_1 + K_1*S_0*k_2 + k_1*k_2)

In [22]:
fps[0].valueDct[A_3]

K_1*K_2*K_3*S_0*T_1*T_2*T_3/(K_1*K_2*K_3*S_0*T_1*T_2 + K_1*K_2*S_0*T_1*k_3 + K_1*S_0*k_2*k_3 + k_1*k_2*k_3)

In [27]:
fps[0].valueDct[A_4]

K_1*K_2*K_3*K_4*S_0*T_1*T_2*T_3*T_4/(K_1*K_2*K_3*K_4*S_0*T_1*T_2*T_3 + K_1*K_2*K_3*S_0*T_1*T_2*k_4 + K_1*K_2*S_0*T_1*k_3*k_4 + K_1*S_0*k_2*k_3*k_4 + k_1*k_2*k_3*k_4)

In [28]:
modelSig.jacobianMat

Matrix([
[  -K_1*S_0 - k_1,                0,                0,              0],
[K_2*(-A_2 + T_2),   -A_1*K_2 - k_2,                0,              0],
[               0, K_3*(-A_3 + T_3),   -A_2*K_3 - k_3,              0],
[               0,                0, K_4*(-A_4 + T_4), -A_3*K_4 - k_4]])

**Observations**
1. Stable system with real eigenvectors.
1. If $K_i >> k_i$, then $A_{i+1} / A_{i} \approx T_{i+1}$ and so $A_n / S_0 \approx \prod_i T_i$

# Architecture II: No M

In [ ]:
def init2Equations():
  statements = [
      "dA = K_A * N - Kd_A * A * P",
      "dP = K_P * A - k_0",
      "equations = [dA, dP]",
  ]
  return "; ".join(statements)

In [ ]:
# Case 2(a): P inhibits A
if False:
    exec(init2Equations())
    sympy.solve([dP], A)
    # How much protein is required
    newdA = dA.subs(A, k_0/K_P)
    sympy.solve(newdA, P)

# Tracking Control

In [ ]:
# Can still get integral control with unimolecular degradation for A, M in A. Why biomolecular in model?
exec(initEquations())
dA = K_A * N - Kd_A * (A + M)
#
sympy.solve([dP], A)

Architecture: P degradation depends on tracking molecule, but it cannot change the level of the molecule.

In [ ]:
MODEL = """
P -> A; KP_1 * P * M
A -> P; -KA_1 * A
A ->; KA * $M
// E -> A; KE * A

$M = 1
KP_1 = 1
KP_2 = 1
KA_1 = 1
KA = 1
KE = 1
"""

rr = te.loada(MODEL)
rr.plot(rr.simulate())

In [ ]:
def init3Equations():
  statements = [
      "dA = K_A * N - Kd_A * A * P",
      "dP = K_P * A - k_0 * M",
      "dM = -k_0 + K_Mp * Mp", 
      "dMp = K_M * M - K_Mp * Mp",
      "equations = [dA, dP]",
  ]
  return "; ".join(statements)

In [ ]:
exec(init3Equations())
sympy.solve([dP], A)

In [ ]:
exec(init3Equations())
sympy.solve(equations, A)

# Integral Control Paper 1

[Feedback Designs](https://drive.google.com/file/d/1mP0A5jvdlVCtdFCy6QbapDgcsWC_LzmU/view?usp=sharing)

The system including $P$ phosphorylation and dephosphorylation.

* $\dot{S} = k_0 x_0 - k_2 P  + k_6 E$
* $\dot{P} = k_4 x_0 - k_7 P + k_8 Pp$. 
* $\dot{Pp} = k_7 P - k_8 Pp$
* $\dot{x}_1 = k_5 S$

**LaPlace Transforms.**

$s S(s) = k_0 X_0 (s) - k_2 P(s) +k_6 E(s)$

$s P(s) = k_4 X_0 (s) -k_7 P(s) + k_8 Pp(s)$

$s Pp(s) = k_7 P(s) - k_8 Pp(s)$
Or, $Pp(s) = P(s) \frac{k_7}{s + k_8}.$

So,
\begin{align*}
sP(s) & = & k_4 X_0 (s) -k_7 P(s) + k_8 Pp(s) \\
& = & k_4 X_0 (s) -k_7 P(s) + k_8 P(s) \frac{k_7}{s + k_8} \\
P(s)& = & X_0 (s) \frac{k_4 }{s + k_7 - \frac{k_7 k_8 }{s + k_8}}
\end{align*}

With this,
\begin{align*}
S(s) = X_0(s) \frac{ k_0 - k_2 \frac{k_4 }{s + k_7 - \frac{k_7 k_8 }{s + k_8}}}{s + k_7} +  E(s) \frac{k_6}{s + k_3}
\end{align*}

\begin{align*}
X_1(s) & = & \frac{k_5}{s}S(s) \\
& = & X_0(s) \frac{k_5}{s}  \frac{ k_0  - k_2 \frac{k_4 }{s + k_7 - \frac{k_7 k_8 }{s + k_8}}}{s + k_7} +  E(s) \frac{k_5}{s} \frac{k_6}{s + k_3}
\end{align*}

This suggests that the impulse response is finite but the step response is unbounded. Also, $x_t(t) \neq 0$ for a non-zeror $e(t)$.

Note that $k_8$ must be small for the integrator to work.
So,
\begin{align*}
X_1(s) & \approx & X_0(s) \frac{k_5}{s}  \frac{ k_0 - k_2 \frac{k_4 }{s + k_7 }}{s + k_7} +  E(s) \frac{k_5}{s} \frac{k_6}{s + k_3}
\end{align*}

Observe that for an impulse $x_0$, 
$x_1 (\infty) =  k_5 \frac{ k_0 k_7 - k_2 k_4}{ k_7^2} .$

Solve for $H_E (s) = \frac{X_1(s)}{E(s)}$.

$X_0(s) = 0$

$s S(s) =  - k_3 S(s) + k_6 E(s)$ And so,
$S(s) = \frac{k_6 E(s)}{s + k_3}$

Further,
$X_1(s) = \frac{k_5}{s}\frac{k_6 E(s)}{s + k_3} .$

Suggest that there is an error with the impulse response, and the system does not converge with the step response.

Solve for transfer function: $H(s) = \frac{X_1(s)}{X_0(s)}$.

$s S(s) = X_0 (s) \frac{k_0 - k_2 k_4 }{s + k_3}$

$ 
\begin{align*}
X_1 (s) & = &  \frac{k_5}{s} S(s) \\
& = & X_0 (s) \frac{k_5}{s^2}  \frac{k_0 - k_2 k_4 }{s + k_3} \\
H(s) & = & \frac{1}{s^2} \frac{k_0 - k_2 k_4 }{s + k_3}
\end{align*}
$

$H (0)$ is constant and so the output tracks the input. However, the system has complex dynamics.

In [ ]:
# Simulation
MODEL = """
$X0 -> S; k0*X0 - k2*P
S -> X1; k4*X0
S -> P; k4*X0
P -> Pp; k7*P
Pp -> P; k8*Pp

$X0 = 1
k0 = 1
k2 = 1
k3
"""

# Integral Paper 2
Analysis of Herbert's early draft.

In [ ]:
su.addSymbols("P S S_1 S_2 _3")

## Model

* $P \xrightarrow{v_r} S$ has kinetics $v_r (P, E_r) = \frac{k_r E_r P}{K_{M_{r}} + P}$
* $S \xrightarrow{v_f} P$ has kinetics $v_f (S, E_f)
= k_f E_f$ because $S$ saturates the reaciton.

Consider region where $K_{M_r} >> P$ and $E_r$ is stable.
Then, $v_r(P, E_r) \approx k_r E_r P$.

The LaPlace transforms are:
* $\frac{S(s)}{P(s)} = \frac{1}{s} k_r E_r$
* $\frac{S(s)}{E(s)} = \frac{T}{s^2} -  \frac{1}{s} k_f E_f(s)$